# **Notebook 4 - Simplified Model (Controlled Simulations)**

---

## **Overview**
This notebook reproduces the **controlled simulations** used to isolate the causal effects of mobility attractors on potential route diversification.  
By modeling an artificial city as a **regular grid (lattice)**, it allows us to systematically vary the **position**, **number**, and **speed limits** of attractors while keeping all other network properties constant.

For clarity, only the core simulation logic is presented here, but all configuration parameters (e.g., grid size, attractor placement, and speed ratios) are easily customizable within the accompanying code.

These experiments confirm the mechanisms observed in real cities and demonstrate how the placement and tuning of mobility attractors affect DiverCity both locally and globally. The outputs correspond to the results shown in *Figure 5* of the paper.

---

## **Parameters**

The table below summarizes the key parameters that can be adjusted to reproduce or extend the simulations.

| Parameter                  | Description                                                                         |
| -------------------------- | ----------------------------------------------------------------------------------- |
| `k`                        | Maximum number of alternative paths considered in DiverCity computation.            |
| `list_p`                   | Penalization parameters applied during path generation (simulate route conformity). |
| `list_eps`                 | Tolerance parameters used to define Near-Shortest Routes (NSR).                     |
| `r_list`                   | Radii (in km) from the city center used to sample origin–destination pairs.         |
| `n_samples`                | Number of origin–destination samples per radius.                                    |
| `center`                   | Coordinates (x, y) for the network center (default is (0, 0)).                      |
| `n_rows`                   | Number of rows in the grid network.                                                 |
| `n_cols`                   | Number of columns in the grid network.                                              |
| `cell_size_km`             | Physical size (km) of each grid cell.                                               |
| `edge_speed_kmh`           | Default speed (km/h) on regular edges.                                              |
| `edge_attractor_speed_kmh` | Speed (km/h) on attractor edges (e.g., main roads, bridges).                        |
| `km_sq_attractors`         | Radii (km) defining square-shaped attractor zones centered on the city.             |
| `rows_attractors`          | Row indices or distances (in km) where horizontal corridors act as attractors.      |
| `cols_attractors`          | Column indices or distances (in km) where vertical corridors act as attractors.     |
| `water_columns`            | Column indices representing impassable water bodies (e.g., rivers).                 |
| `bridges_at_rows`          | Row indices where bridge crossings are allowed across water columns.                |
| `max_it`                   | Maximum number of iterations for the Path Penalization algorithm.                   |


In [ ]:
import matplotlib.pyplot as plt
from simplified_model import launch_simplified_model, plot_grid_map
import numpy as np
from my_utils import set_mystyle

from results_utils import get_divercity_vs_radius_at_k

## **1. Simulation Parameters**

The parameters below define the configuration of the simplified grid-based simulation.  
They control the structure of the artificial network, the properties of mobility attractors,  
and the parameters used for computing DiverCity.

In [ ]:
# ------------------------------------------------------------
# DiverCity Computation Parameters
# ------------------------------------------------------------

k = 10                        # Number of alternative routes (near-shortest paths)
list_p = [0.1]                # Path penalization factors for the Path Penalization (PP) algorithm
list_eps = [0.3]              # Epsilon tolerance for near-shortest routes (±30% of the fastest route)
r_list = np.arange(1, 31, 1)  # Radial distances from the grid center (1–30 km)
n_samples = 5                 # Number of origin–destination pairs per radial distance

# ------------------------------------------------------------
# Grid Network Configuration
# ------------------------------------------------------------

n_rows = 120                  # Number of rows in the lattice grid
n_cols = 120                  # Number of columns in the lattice grid
cell_size_km = 0.5            # Cell size (edge length) in kilometers

# Default speed limits (baseline scenario)
edge_speed_kmh = 50           # Speed limit for regular roads (km/h)
edge_attractor_speed_kmh = 100 # Speed limit for attractor edges (km/h)

## **2. Experiments**

The following experiments demonstrate how the simplified model can be used to study  
the impact of **mobility attractors** on potential route diversification in a controlled environment.  

Each example represents a different attractor configuration:
- **No attractors** → Baseline grid with uniform speed limits.  
- **1 attractor** at 10 km → Simulates a single high-speed corridor (e.g., a ring road).  
- **2 attractors** at 10 km and 11 km → Tests the stabilizing effect of clustered attractors.  

These examples are provided as reference experiments to **illustrate how the model works**.  
Users can easily customize parameters (e.g., attractor positions, speeds, grid size, or sampling)  
to explore additional scenarios and sensitivity analyses.

⚠️ Note: This step may take some time to complete, depending on the network size, the number of origin–destination pairs, and the number of parallel jobs (`njobs`).

In [ ]:
# ------------------------------------------------------------
# Run example experiments with different attractor configurations
# ------------------------------------------------------------

dict_result_attractors = {}

for attractors in [[], [10], [10, 11]]:
    print(f"\nRunning simulation with attractors at: {attractors if attractors else 'none'}")

    G, df_result, penalized_paths_dict = launch_simplified_model(
        k,
        list_p,
        list_eps,
        r_list,
        n_samples=4,  # number of OD pairs per radius
        n_rows=n_rows,
        n_cols=n_cols,
        cell_size_km=cell_size_km,
        edge_speed_kmh=edge_speed_kmh,
        edge_attractor_speed_kmh=edge_attractor_speed_kmh,
        km_sq_attractors=attractors,
        max_it=200,
        njobs=10
    )

    # Store results for comparison
    dict_result_attractors[tuple(attractors)] = df_result

#### Example: plotting the grid

In [ ]:
plot_grid_map(G, linewidth_attr=2)

## **3. Plot DiverCity vs. Radius**

This plot illustrates how **DiverCity** varies with distance from the city center under different attractor configurations.  
For each scenario (no attractors, one attractor, two attractors), the DiverCity profile is computed as the median  
across sampled origin–destination pairs at each radial distance.

The resulting curves replicate the trends observed in the paper (*Figure 5*), showing:
- A rapid increase in DiverCity near the center, followed by stabilization at larger distances.  
- A localized drop in DiverCity in the vicinity of mobility attractors.  
- A global stabilization effect when multiple attractors are clustered.

This analysis highlights how attractor configuration influences both local and city-wide route diversification.


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

for attractors in dict_result_attractors:
    y_vector = get_divercity_vs_radius_at_k(dict_result_attractors[attractors], k, p=0.1, eps=0.3)
    ax.plot(np.arange(1, len(y_vector) + 1, 1),y_vector, alpha=1, linewidth=1.5, label=attractors)

plt.legend(title="Attractors (km)")
ax.set_ylabel(r"${\mathcal{D}(u, v)}$", fontsize=10, fontweight=400)
ax.set_xlabel("radius [km]", fontsize=10, fontweight=400)
set_mystyle(ax)

## **4. Example: Grid with a Bridge**

This experiment introduces a **bridge-like bottleneck** into the simplified grid model.  
It is an **illustrative example** designed to simulate the effect of a natural or infrastructural constraint (e.g., a river or bay) that divides the city into two disconnected regions,  
connected only by one or more bridges.

The parameter `water_columns` specifies the columns representing the impassable area (the “river”),  
while `bridges_at_rows` defines where bridges are placed to reconnect the two sides.

This setup allows testing how such bottlenecks influence DiverCity and travel times,  
particularly under speed reduction experiments, replicating the patterns observed in real bridge-dominated cities (e.g., *San Francisco* or *Rio de Janeiro*).

⚠️ Note: This step may take some time to complete, depending on the network size, the number of origin–destination pairs, and the number of parallel jobs (`njobs`).

In [ ]:
# ------------------------------------------------------------
# Run illustrative example: grid with a bridge-like bottleneck
# ------------------------------------------------------------

# Define the "river" region and bridge positions
water_columns = list(np.arange(-5, 5.5, 0.5))  # Columns representing the water gap
bridges_at_rows = [0]                          # Rows where bridges connect the two sides

# Launch the simplified model with bridge configuration
G, df_result, penalized_paths_dict = launch_simplified_model(
    k,
    list_p,
    list_eps,
    r_list,
    n_samples=4,
    n_rows=n_rows,
    n_cols=n_cols,
    cell_size_km=cell_size_km,
    edge_speed_kmh=edge_speed_kmh,
    edge_attractor_speed_kmh=edge_attractor_speed_kmh,
    water_columns=water_columns,
    bridges_at_rows=bridges_at_rows,
    km_sq_attractors=[],
    max_it=200,
    njobs=10
)

In [ ]:
# visualize the grid
plot_grid_map(G, linewidth_attr=2)